In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [17]:
#from google.colab import drive
#drive.mount('/content/drive')
torch.cuda.is_available()

False

In [18]:
def get_chest_images():
    TRANSFORM_IMG = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor(), ])
    file_path = "/storage/pipemon/6.867-xray-project/data/"
    #define train data loader
    trainset = datasets.ImageFolder(root=file_path+"train/", transform=TRANSFORM_IMG)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle=True, num_workers=2)
    #define test data loader
    valset = datasets.ImageFolder(root=file_path+"test/", transform=TRANSFORM_IMG)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=False, num_workers=2)
    return train_loader, val_loader
    

In [19]:
class Chest_Disease_Net(nn.Module):
    def __init__(self):
        super(Chest_Disease_Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=16, stride=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=7, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(12544, 512),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(512,2)
         
    def forward(self, x):
        print(x.shape)
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
        out = self.layer3(out)
        print(out.shape)
        out = self.layer4(out)
        print(out.shape)
        out = self.layer5(out)
        print(out.shape)
        out = out.reshape(out.size(0), -1)
        print(out.shape)
        out = self.fc1(out)
        print(out.shape)
        out = self.fc2(out)
        print(out.shape)
        out = self.fc3(out)
        print(out.shape)
        return(out)

In [20]:
def run():
    # Parameters
    num_epochs = 10
    output_period = 100
    batch_size = 100

    # setup the device for running
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device: ", device)
    model = Chest_Disease_Net()
    model = model.to(device)

    train_loader, val_loader = get_chest_images()
    num_train_batches = len(train_loader)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    epoch = 1
    while epoch <= num_epochs:
        running_loss = 0.0
        for param_group in optimizer.param_groups:
            print('Current learning rate: ' + str(param_group['lr']))
        model.train()

        for batch_num, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if batch_num % output_period == 0:
                '''
                print('[%d:%.2f] loss: %.3f' % (
                    epoch, batch_num*1.0/num_train_batches,
                    running_loss/output_period
                    ))
                '''
                running_loss = 0.0
                gc.collect()

        gc.collect()
        # save after every epoch
        torch.save(model.state_dict(), "models/model.%d" % epoch)


        val_correct = 0
        correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                _, predicted_five = torch.topk(outputs.data, 5, dim=1)

                val_total += labels.size(0)

                correct += (predicted == labels).sum().item()
                val_correct += (predicted_five[:, 0] == labels).sum().item()
                val_correct += (predicted_five[:, 1] == labels).sum().item()
                val_correct += (predicted_five[:, 2] == labels).sum().item()
                val_correct += (predicted_five[:, 3] == labels).sum().item()
                val_correct += (predicted_five[:, 4] == labels).sum().item()
        print('Top One Error of the network on validation images: %d %%' % (
           100 * (1 - correct/val_total)))
        print('Top Five Error of the network on validation images: %d %%' % (
           100 * (1 - val_correct / val_total)))

        gc.collect()
        epoch += 1

run()

Device:  cpu


  0%|          | 0/21631 [00:00<?, ?it/s]

Current learning rate: 0.001
torch.Size([4, 3, 256, 256])


  0%|          | 0/21631 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size 64 1 16 16, expected input[4, 3, 256, 256] to have 1 channels, but got 3 channels instead